In [ ]:
pip install pymongo django django.http django.db

**Predict and Similiarity score using audio input**

views.py

In [ ]:
from django.http import JsonResponse
from .audio_utils import preprocess_audio, predict_from_features
from keras.models import load_model

# Load the model on startup
model = load_model('my_model.keras')

def predict_disease(request):
    if request.method == 'POST':
        audio_file = request.FILES.get('audio', None)
        if audio_file:
            try:
                features = preprocess_audio(audio_file)
                predictions = predict_from_features(model, features)
                return JsonResponse({'predictions': predictions.tolist()}, status=200)
            except Exception as e:
                return JsonResponse({'error': str(e)}, status=500)
        else:
            return JsonResponse({'error': 'No audio file provided'}, status=400)
    else:
        return JsonResponse({'error': 'Invalid request method'}, status=405)

audio_utils.py

In [ ]:
import librosa
import numpy as np
import noisereduce as nr

def preprocess_audio(audio_file):
    audio, sr = librosa.load(audio_file, sr=None)
    audio = nr.reduce_noise(y=audio, sr=sr)
    audio = librosa.util.normalize(audio)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfccs = np.mean(mfccs.T, axis=0)
    return mfccs

def predict_from_features(model, features):
    return model.predict(np.array([features]))

urls.py

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path('predict/', views.predict_disease, name='predict_disease'),
]

In [ ]:
from django.conf import settings
from django.conf.urls.static import static

urlpatterns += static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)

settings.py

In [ ]:
MEDIA_URL = '/media/'
MEDIA_ROOT = os.path.join(BASE_DIR, 'media')

**Incremental Learning Prep**

In [ ]:
from pymongo import MongoClient
import librosa

# Connect to MongoDB
client = MongoClient('mongodb_uri')
db = client.respiratory_diseases
audio_collection = db.audio_data

def store_audio_data(audio_file, patient_info, annotations):
    audio, sr = librosa.load(audio_file, sr=None)
    audio_features = extract_features(audio, sr)
    
    # Create a document to store in MongoDB
    document = {
        'patient_id': patient_info['patient_id'],
        'audio_features': audio_features,
        'annotations': annotations
    }
    
    # Insert document into MongoDB
    audio_collection.insert_one(document)

def extract_features(audio, sr):
    # Implement feature extraction, e.g., MFCCs
    mfccs = librosa.feature.mfcc(audio, sr=sr)
    return mfccs.tolist()  # Convert to list for MongoDB compatibility

# Example usage
audio_file = 'path_to_audio_file.wav'
patient_info = {'patient_id': 101}
annotations = {'crackles': 0, 'wheezes': 1}
store_audio_data(audio_file, patient_info, annotations)

**MongoDB**  

Database Schema Design

**Patients**

**AudioRecordings**

**Annotations**

**Diagnoses**

*Patients Collection*

This collection stores demographic information. Each document in the collection might look like this:

In [ ]:
{
  "_id": ObjectId("..."),
  "patientNumber": 101,
  "age": 45,
  "sex": "Male",
  "bmi": 22.5,
  "childWeight": null,
  "childHeight": null
}

*AudioRecordings Collection*

This collection stores references to audio files along with metadata from the filename parsing:

In [ ]:
{
  "_id": ObjectId("..."),
  "patientNumber": 101,
  "recordingIndex": 1,
  "chestLocation": "Trachea",
  "acquisitionMode": "sc",
  "recordingEquipment": "AKGC417L",
  "filePath": "path/to/audio/file.wav"
}

*Annotations Collection*

Stores details about the respiratory cycle and presence of crackles or wheezes:

In [ ]:
{
  "_id": ObjectId("..."),
  "audioRecordingId": ObjectId("reference to AudioRecordings"),
  "beginCycle": 0.5,
  "endCycle": 2.5,
  "crackles": 1,
  "wheezes": 0
}

*Diagnoses Collection*

Stores the diagnosed conditions for patients:

In [ ]:
{
  "_id": ObjectId("..."),
  "patientNumber": 101,
  "diagnosis": ["COPD"]
}

**Indexing**

Create indexes on frequently queried fields to improve performance:

Patients: *patientNumber*

AudioRecordings: Composite index on *patientNumber*, *chestLocation*

Annotations: *audioRecordingId*

Diagnoses: *patientNumber*

Setting up

In [ ]:
pip install djongo

settings.py

In [ ]:
DATABASES = {
    'default': {
        'ENGINE': 'djongo',
        'NAME': 'your_database_name',   # Name of your MongoDB database
        'ENFORCE_SCHEMA': False,  # Djongo specific option to not enforce Django schema on MongoDB
        'CLIENT': {
            'host': 'mongodb://localhost:27017',  # URL to your MongoDB server
            'username': 'your_username',  # MongoDB username (if authentication is enabled)
            'password': 'your_password',  # MongoDB password
            'authMechanism': 'SCRAM-SHA-1'  # Authentication mechanism (e.g., SCRAM-SHA-1, SCRAM-SHA-256)
        }
    }
}

**Migrations**

In [ ]:
python manage.py makemigrations
python manage.py migrate

Check

In [ ]:
python manage.py runserver